# Extraer palabras claves identificadas en textos
Se implementan múltiples modelos para la extracción de características como palabras clave de textos a fin de mejorar el desempeño de la comparación semántica entre productos similares

## Importes Iniciales

In [1]:
%load_ext IPython.extensions.autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

def find_src_folder(current_path: Path, folder_name: str = 'src') -> Path:
    search_directories = [current_path] + list(current_path.parents)
    for parent in search_directories:
        if parent.name == folder_name:
            return parent.parent
    return current_path

src_path = find_src_folder(Path.cwd(), 'src')
sys.path.append(str(src_path))

In [3]:
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

import pandas as pd

from matplotlib.ticker import FuncFormatter

import tensorflow_hub as hub

from sklearn.neighbors import NearestNeighbors

from langchain_ollama.embeddings import OllamaEmbeddings

from sklearn.cluster import DBSCAN
from pyspark.sql import functions as F, types as T

from src.utils import SparkUtils
from keybert import KeyBERT

2025-07-06 18:17:32.495767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751843852.715387  166318 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751843852.778174  166318 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-06 18:17:33.378475: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/mnt/c/Users/User/Documents/Maestría/Amazon Reviews Code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IPr

In [4]:
spark_utils = SparkUtils('expand_keywords')

your 131072x1 screen size is bogus. expect trouble
25/07/06 06:57:41 WARN Utils: Your hostname, ASUSPRIMEA520MAII resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/06 06:57:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/User/Documents/Maestr%c3%ada/Amazon%20Reviews%20Code/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/edgar/.ivy2/cache
The jars for the packages stored in: /home/edgar/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ab46b37-5559-4259-a072-49b8765ec077;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 239ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

In [5]:
spark = spark_utils.spark

## Generar modelo extractor

In [ ]:
model = KeyBERT('all-MiniLM-L6-v2')

In [25]:
model.extract_keywords(pd.Series(['ola de mar', 'gg', '']))

[[('ola', 0.585), ('mar', 0.5813)], [('gg', 1.0)], []]

In [31]:
@F.pandas_udf(T.ArrayType(T.StringType()))
def extract_keywords(text_series: pd.Series) -> pd.Series:
    return text_series.apply(lambda text: [
        kw[0] for kw in model.extract_keywords(
            text,
            keyphrase_ngram_range=(1, 2),
            stop_words='english',
            top_n=5
        )
    ] if text else [])

In [32]:
meta_items_texts = spark.read.parquet(spark_utils.path('meta_items_description_and_features_unified'))

In [33]:
meta_items_texts_with_keywords =(
    meta_items_texts.limit(100)
        .withColumn(
            'keywords_description', extract_keywords(F.col('description'))
        )
        .withColumn(
            'features_description', extract_keywords(F.col('features'))
        )
)

In [35]:
(
    meta_items_texts_with_keywords
        .write
        .format('delta')
        .mode('overwrite')
        .save(spark_utils.path('meta_items_with_keywords'))
)

2025-07-06 08:10:56.839388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-06 08:10:56.839725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751807456.857953   36676 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751807456.858206   36682 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751807456.863530   36676 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
E0000 00:00:1751807456.863821   36682 cuda_blas.cc:1

In [36]:
meta_items_leywords = spark.read.parquet(spark_utils.path('meta_items_with_keywords'))

In [39]:
meta_items_leywords.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------+-------------+-----+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+
|               title|       main_category|            features|         description|average_rating|rating_number|price|               store|parent_asin|categories|             details|              images|keywords_description|features_description|
+--------------------+--------------------+--------------------+--------------------+--------------+-------------+-----+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+
|  Accupressure Guide|Appstore for Android|All the pressing ...|Acupressure techn...|           3.6|         NULL|  0.0|           mAppsguru| B00VRPSGEO|        []|{"Release Date":"...|[{"large":"https:...|[acupressure tech...|[pressing point, ...|
|Ank